In [36]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [54]:
# Negative Sample
neg_sample_visibility = pd.read_csv('../data/processed/neg_sample_visibility.csv', index_col=[0])
neg_sample_temperature = pd.read_csv('../data/processed/neg_sample_temperature.csv', index_col=[0])
neg_sample_precipitation = pd.read_csv('../data/processed/neg_sample_precipitation.csv', index_col=[0])
negative_sample = pd.read_csv('../data/processed/negative_sample.csv', index_col=[0])

# Positive sample
pos_sample_visibility = pd.read_csv('../data/processed/pos_sample_visibility.csv', index_col=[0])
pos_sample_temperature = pd.read_csv('../data/processed/pos_sample_temperature.csv', index_col=[0])
pos_sample_precipitation = pd.read_csv('../data/processed/pos_sample_precipitation.csv', index_col=[0])
positive_sample = pd.read_csv('../data/processed/positive_sample.csv', index_col=[0])


In [55]:
# Merge
negative_sample['humidity'], negative_sample['temparature'], negative_sample['visibility'],negative_sample['prec_height'],negative_sample['prec_duration']  =  neg_sample_temperature['humidity'], neg_sample_temperature['temperature'], neg_sample_visibility['visibility'],neg_sample_precipitation['prec_height'],neg_sample_precipitation['prec_duration'] 
positive_sample['humidity'], positive_sample['temparature'], positive_sample['visibility'],positive_sample['prec_height'],positive_sample['prec_duration']  =  pos_sample_temperature['humidity'], pos_sample_temperature['temperature'], pos_sample_visibility['visibility'],pos_sample_precipitation['prec_height'],pos_sample_precipitation['prec_duration'] 

full_data = negative_sample.append(positive_sample)
full_data.columns

Index(['LINREFX', 'LINREFY', 'LOR', 'LOR_ab_2021', 'XGCSWGS84', 'YGCSWGS84',
       'acc_cat', 'acc_type1', 'acc_type2', 'bike', 'car', 'col_id',
       'collision', 'collision_cnt', 'district', 'foot', 'geometry', 'hour',
       'hour_cos', 'hour_sin', 'humidity', 'land', 'length_m', 'lightratio',
       'mid_lat', 'mid_lon', 'month', 'month_cos', 'month_sin', 'motor',
       'objectid', 'other', 'prec_duration', 'prec_height', 'road_con',
       'segment_id', 'side_strt', 'street', 'sun_elevation_angle',
       'temparature', 'vehicle', 'visibility', 'weekday', 'year'],
      dtype='object')

In [56]:
# FUll DATA
full_data = full_data[[
    'segment_id',
    'year',
    'month_cos',
    'month_sin',
    'weekday',
    'hour_cos',
    'hour_sin',
    'collision_cnt',
    'side_strt',
    'sun_elevation_angle',
    'humidity',
    'temparature',
    'visibility',
    'collision'
]]

full_data['collision_cnt'] = full_data['collision_cnt'].fillna(0)

full_data = full_data.round(3)

#full_data['weekday'] = full_data['weekday'].astype('category')

full_data

In [60]:
full_data

,segment_id,year,month_cos,month_sin,weekday,hour_cos,hour_sin,collision_cnt,side_strt,sun_elevation_angle,humidity,temparature,visibility,collision
0,42796.0,2019,0.866,-0.5,6,0.963,0.270,0.0,1.0,-53.638,86.187,4.081,20049.462,0
1,34322.0,2020,0.866,0.5,6,0.460,0.888,0.0,1.0,-36.877,88.226,3.458,23919.355,0
2,29497.0,2018,-1.000,0.0,2,-0.776,0.631,1.0,1.0,35.339,55.533,20.693,34946.667,0
3,39002.0,2020,-0.000,-1.0,6,-0.335,0.942,0.0,1.0,1.920,76.000,14.227,36333.333,0
4,32881.0,2019,-0.866,-0.5,2,1.000,0.000,0.0,1.0,-14.045,72.806,17.171,40709.677,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52113,21737.0,2020,1.000,-0.0,6,-0.991,-0.136,6.0,0.0,14.374,77.194,5.045,24145.161,1
52114,2782.0,2020,1.000,-0.0,2,-0.068,-0.998,20.0,0.0,-9.363,82.355,4.119,24951.613,1
52115,2781.0,2020,1.000,-0.0,5,-0.917,0.398,21.0,0.0,9.907,81.161,3.971,22723.333,1
52116,19720.0,2020,1.000,-0.0,3,-0.577,-0.817,14.0,0.0,5.171,79.516,4.648,22467.742,1


In [61]:
# Split
X_train = full_data[full_data['year'] < 2020].drop(['collision'],axis=1)
X_test = full_data[full_data['year'] == 2020].drop(['collision'],axis=1)

y_train = full_data[full_data['year'] < 2020]['collision']
y_test = full_data[full_data['year'] == 2020]['collision']

In [62]:
y_train.value_counts()

0    174119
1     36227
Name: collision, dtype: int64

In [63]:
# Random Forest
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector

num_pipe = SimpleImputer(strategy="mean", add_indicator=True)
cat_pipe = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor_tree = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

rf_clf = make_pipeline(
    preprocessor_tree, RandomForestClassifier(
        n_estimators=70
    )
)

In [64]:
cv_result = cross_validate(rf_clf, X_train, y_train, scoring="roc_auc", cv=5)

NameError: name 'cross_validate' is not defined

In [103]:
cv_result['test_score'].mean()

0.8611297292666377

In [104]:
rf_clf.fit(X_train, y_train).steps[1][1].feature_importances_

array([0.00726219, 0.01009037, 0.0111664 , 0.03262527, 0.02704504,
       0.40386322, 0.04819241, 0.2251177 , 0.06940659, 0.06476094,
       0.02758721, 0.00930434, 0.01043333, 0.01084651, 0.0113433 ,
       0.01118425, 0.01085752, 0.00891341])

In [110]:
# Plot Feature Importance for baseline RF model

import time
import numpy as np

forest_importances = pd.Series(rf_clf.steps[1][1].feature_importances_, index=rf_clf.feature_names_in_)

start_time = time.time()
importances = rf_clf.steps[1][1].feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_clf.steps[1][1].estimators_], axis=0)
elapsed_time = time.time() - start_time

fig, ax = plt.subplots()
forest_importances.sort_values(axis=0, ascending=False).plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")

ValueError: Length of passed values is 18, index implies 12

In [1]:
## RANDOM PARAMETER GRID SEARCH
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(200, 400, 100)]
# Number of features to consider at every split
max_features = ['log2','sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(30, 50, 10)]
max_depth.append(None)

criterion = ['gini', 'entropy']
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split':min_samples_split,
               'min_samples_leaf':min_samples_leaf,
               'bootstrap': bootstrap,
                'criterion':criterion}

print(random_grid)

{'n_estimators': [200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373, 375, 377, 379, 381, 383, 385, 387, 389, 391, 393, 395, 397, 400], 'max_features': ['log2', 'sqrt'], 'max_depth': [30, 32, 34, 36, 38, 41, 43, 45, 47, 50, None], 'min_samples_split': [2], 'min_samples_leaf': [1], 'bootstrap': [False], 'criterion': ['gini', 'entropy']}


In [254]:
rf_random = GridSearchCV(estimator = RandomForestClassifier(), 
                               param_grid = random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [255]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 1920 candidates, totalling 5760 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [30, 35, 40, 45, 50, None],
                         'max_features': ['auto'],
                         'min_samples_leaf': [1, 6, 11, 16],
                         'min_samples_split': [2, 5, 8, 12],
                         'n_estimators': [250, 261, 272, 283, 294, 305, 316,
                                          327, 338, 350]},
             return_train_score=True, scoring='roc_auc', verbose=2)

In [256]:
rf_random.best_score_

0.8956388086414308

In [257]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 45,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 327}

In [ ]:
brf_random = GridSearchCV(estimator = BalancedRandomForestClassifier(), 
                               param_grid = random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [ ]:
brf_random.fit(X_train, y_train)

In [ ]:
brf_random.best_score_

In [ ]:
brf_random.best_params_

In [2]:
#Random Forest with oversampling using SMOTE and undersampling using RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline

from collections import Counter

# summarize class distribution - highly imbalanced dataset
counter = Counter(y_train)
print(counter)


## RANDOM PARAMETER GRID SEARCH
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

over_n = [float(x) for x in np.arange(0.25, 0.35, 0.10)]
# RandomUnderSampler undersampling
under_n = [float(x) for x in np.arange(0.35, 0.45, 0.10)]

# Create the random grid
random_grid = {'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__max_depth': max_depth,
               'randomforestclassifier__min_samples_split': min_samples_split,
               'randomforestclassifier__min_samples_leaf': min_samples_leaf,
               'randomforestclassifier__bootstrap': bootstrap,
               'randomforestclassifier__criterion':criterion,
              'smote__sampling_strategy':over_n,
              'randomundersampler__sampling_strategy':under_n}

print(random_grid)

NameError: name 'y_train' is not defined

In [ ]:
# define pipeline
# oversample positive (minority) to be x% the number of negative (majority)
over = SMOTE(sampling_strategy = 0.1)
# randomly undersample negative (majority) to reduce the number of negative to x% of the positive (minority)
under = RandomUnderSampler(sampling_strategy = 0.25) 

pipeline = make_pipeline(over, under, RandomForestClassifier(n_estimators = 260,
 min_samples_split =2,
 min_samples_leaf=1,
 max_features= 'auto',
 max_depth= 90,
 bootstrap= False))

rf_sampling_random = GridSearchCV(pipeline,
                               param_grid = random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [ ]:
rf_sampling_random.fit(X_train,y_train)
rf_sampling_random.best_score_


In [ ]:
rf_sampling_random.best_params_

In [472]:
# BALANCED BAGGING

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 150, stop = 300, num = 5)]

# Method of selecting samples for training each tree
bootstrap = [True, False]

over_n = [float(x) for x in np.arange(0.1, 0.35, 0.05)]
# RandomUnderSampler undersampling
under_n = [float(x) for x in np.arange(0.35, 0.5, 0.05)]

# Create the random grid
bb_random_grid = {'balancedbaggingclassifier__n_estimators': n_estimators,
                 'balancedbaggingclassifier__bootstrap': bootstrap,
                 'smote__sampling_strategy':over_n,
              'randomundersampler__sampling_strategy':under_n}


# define pipeline
# oversample positive (minority) to be x% the number of negative (majority)
over = SMOTE(sampling_strategy = 0.1)
# randomly undersample negative (majority) to reduce the number of negative to x% of the positive (minority)
under = RandomUnderSampler(sampling_strategy = 0.25) 

pipeline = make_pipeline(over, under, BalancedBaggingClassifier(base_estimator=HistGradientBoostingClassifier(random_state=42)))

In [473]:
bb_random = GridSearchCV(estimator = pipeline, 
                               param_grid = bb_random_grid,
                               scoring='roc_auc',
                               cv = 3, 
                               verbose=2, 
                               n_jobs = -1,
                               return_train_score = True)

In [250]:
bb_random.fit(X_train, y_train)
bb_random.best_score_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


0.8569596662369879

In [234]:
bb_random.best_params_

{'n_estimators': 207, 'bootstrap': True}

In [106]:
# LOGISTIC REGRESSION
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate



num_pipe = make_pipeline(
    StandardScaler(), SimpleImputer(strategy="mean", add_indicator=True)
)
cat_pipe = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor_linear = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

lr_clf = make_pipeline(preprocessor_linear, LogisticRegression(max_iter=1000))
lr_clf.set_params(logisticregression__class_weight="balanced")

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=2,
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler()),
                                                                  ('simpleimputer',
                                                                   SimpleImputer(add_indicator=True))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000017D03FB0988>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_col

In [107]:
cv_result = cross_validate(lr_clf, X_train, y_train, scoring="roc_auc", cv=5)

In [108]:
cv_result['test_score'].mean()

0.896134662615431

In [477]:
# Support Vector Classification with RandomOverSampling
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline 

num_pipe = make_pipeline(
    MinMaxScaler(feature_range=(0, 1))
)
cat_pipe = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor_svc = make_column_transformer(
    (num_pipe, selector(dtype_include="number")),
    (cat_pipe, selector(dtype_include="category")),
    n_jobs=2,
)

smote = SMOTE(sampling_strategy=0.1)

rus = RandomUnderSampler(sampling_strategy=0.2)

svc_clf = make_pipeline(preprocessor_svc, smote, rus, SVC(kernel='rbf',C=1))

In [478]:
cv_result = cross_validate(svc_clf, X_train, y_train, scoring="roc_auc", cv=5)

In [479]:
cv_result['test_score'].mean()

0.8430180789451789

In [482]:
parameters = {'smote__sampling_strategy':[float(x) for x in np.arange(0.1, 0.35, 0.05)],
              'randomundersampler__sampling_strategy':[float(x) for x in np.arange(0.35, 0.5, 0.05)],
              'svc__kernel':('linear', 'rbf', 'poly'), 
              'svc__C':[1,10,100,1000],
              'svc__gamma':[1,0.1,0.001,0.0001], 
              'svc__degree':[1,2]}

svc_grid = GridSearchCV(svc_clf, parameters, scoring="roc_auc", cv=3)

In [483]:
svc_grid.fit(X_train, y_train)
svc_grid.best_score_

KeyboardInterrupt: 

In [445]:
svc_grid.best_params_

{'rus__sampling_strategy': 0.2,
 'smote__sampling_strategy': 0.1,
 'svc__C': 1,
 'svc__class_weight': None,
 'svc__degree': 2,
 'svc__kernel': 'rbf'}